In [12]:
# Load the Drive helper and mount
from google.colab import drive

# This will prompt for authorization.
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [0]:
import sys,os
import sqlite3
os.chdir("/content/drive/My Drive/Colab Notebooks/zttest/")

In [38]:
pd.read_csv("RA_test.csv").iloc[0]

groupname                                   wells fargo funds trust
submissiontype                                              485APOS
monthend                                                 12/31/2009
link              https://www.sec.gov/Archives/edgar/data/108140...
Name: 0, dtype: object

In [41]:
doc_list = pd.read_csv("RA_test.csv")
doc_list

,groupname,submissiontype,monthend,link
0,wells fargo funds trust,485APOS,12/31/2009,https://www.sec.gov/Archives/edgar/data/108140...
1,strategic partners opportunity funds,485BPOS,2/28/2002,https://www.sec.gov/Archives/edgar/data/110463...
2,sti classic funds,485APOS,7/31/2001,https://www.sec.gov/Archives/edgar/data/883939...
3,ssga funds,485BPOS,12/31/2002,https://www.sec.gov/Archives/edgar/data/826686...
4,mutual fund series trust,485BPOS,10/31/2014,https://www.sec.gov/Archives/edgar/data/135506...
5,mosaic equity trust,485APOS,2/28/2005,https://www.sec.gov/Archives/edgar/data/710977...


In [0]:
import urllib2

In [0]:
txt_file = [];
for each_link in doc_list['link'].values:
    txt_file.append(urllib2.urlopen(each_link).read())

In [45]:
for x in doc_list['link'].values:
    print(type(x),x)

(<type 'str'>, 'https://www.sec.gov/Archives/edgar/data/1081400/000119312509256878/0001193125-09-256878.txt')
(<type 'str'>, 'https://www.sec.gov/Archives/edgar/data/1104631/000095013002001024/0000950130-02-001024.txt')
(<type 'str'>, 'https://www.sec.gov/Archives/edgar/data/883939/000093506901500295/0000935069-01-500295.txt')
(<type 'str'>, 'https://www.sec.gov/Archives/edgar/data/826686/000104746902008442/0001047469-02-008442.txt')
(<type 'str'>, 'https://www.sec.gov/Archives/edgar/data/1355064/000091047214004753/0000910472-14-004753.txt')
(<type 'str'>, 'https://www.sec.gov/Archives/edgar/data/710977/000031040705000015/0000310407-05-000015.txt')


In [56]:
txt_file[3]

Output hidden; open in https://colab.research.google.com to view.

In [57]:
txt_file[4]

Output hidden; open in https://colab.research.google.com to view.

In [58]:
txt_file[5]

'-----BEGIN PRIVACY-ENHANCED MESSAGE-----\nProc-Type: 2001,MIC-CLEAR\nOriginator-Name: webmaster@www.sec.gov\nOriginator-Key-Asymmetric:\n MFgwCgYEVQgBAQICAf8DSgAwRwJAW2sNKK9AVtBzYZmr6aGjlWyK3XmZv3dTINen\n TWSM7vrzLADbmYQaionwg5sDW3P6oaM5D3tdezXMm7z1T+B+twIDAQAB\nMIC-Info: RSA-MD5,RSA,\n DV3iWGcEf9QKJ8uMZPnQx4jm8VdMwPDoP5rgFl5zELN6C8iPJofKuqnnZ6iJyBSr\n q2X2baoWVEO0sXMca+Ck8Q==\n\n<SEC-DOCUMENT>0000310407-05-000015.txt : 20050225\n<SEC-HEADER>0000310407-05-000015.hdr.sgml : 20050225\n<ACCEPTANCE-DATETIME>20050225143532\nACCESSION NUMBER:\t\t0000310407-05-000015\nCONFORMED SUBMISSION TYPE:\t485APOS\nPUBLIC DOCUMENT COUNT:\t\t13\nFILED AS OF DATE:\t\t20050225\nDATE AS OF CHANGE:\t\t20050225\n\nFILER:\n\n\tCOMPANY DATA:\t\n\t\tCOMPANY CONFORMED NAME:\t\t\tMOSAIC EQUITY TRUST\n\t\tCENTRAL INDEX KEY:\t\t\t0000710977\n\t\tIRS NUMBER:\t\t\t\t546200855\n\t\tSTATE OF INCORPORATION:\t\t\tMA\n\t\tFISCAL YEAR END:\t\t\t1231\n\n\tFILING VALUES:\n\t\tFORM TYPE:\t\t485APOS\n\t\tSEC ACT:\t\t1933 Act\n

In [162]:
# -*- coding: utf-8 -*-
"""
Spyder Editor

This is a temporary script file.
"""

import pandas as pd
import numpy as np
import urllib2
import string
import calendar
import sqlite3
from sqlite3 import Error


def text_parser(doc_list):
    groupname_list = []
    submissiontype_list = []
    monthend_list = []
    prospectuses = []
    status = []
    
    for i in range(len(doc_list)):
        groupname_check = False
        submissiontype_check = False
        monthend_check = False
        temp_groupname = ""
        temp_submissiontype = ""
        temp_monthend = ""
        temp_status = "failed "
        
        # Retrive prospectuses of interest
        response = urllib2.urlopen(doc_list['link'].iloc[i])
        html_doc = response.read()
        
        # a. Extract header
        header = html_doc[html_doc.find('<SEC-HEADER>')+len('<SEC-HEADER>'):html_doc.find('</SEC-HEADER>')-1]
        header = header.split('\n')
    #     print(html_doc)
        # b. Extract following information from header
        # groupname
        # submission type
        # monthend
        
        for txt in header:
            if "COMPANY CONFORMED NAME:" in txt:
                temp_groupname = string.lower(txt[txt.rfind('\t')+1:].replace('\t',"").replace('\n',""))
            if "CONFORMED SUBMISSION TYPE:" in txt:
                temp_submissiontype = txt[txt.rfind('\t')+1:].replace('\t',"").replace('\n',"")
            if "FILED AS OF DATE:" in txt:
                temp_date = txt[txt.rfind('\t')+1:].replace('\t',"").replace('\n',"")
                temp_monthend = str(int(temp_date[4:6])) +'/' + str(calendar.monthrange(int(temp_date[:4]),int(temp_date[4:6]))[1]) +'/'+temp_date[:4]
            if temp_groupname != "" and temp_submissiontype != "" and temp_monthend !="":
                break
    #    print (temp_monthend, doc_list['monthend'].iloc[i])
        if temp_groupname == doc_list['groupname'].iloc[i]:
            groupname_check = True
        if temp_submissiontype == doc_list['submissiontype'].iloc[i]:
            submissiontype_check = True
        if temp_monthend == doc_list['monthend'].iloc[i]:
            monthend_check = True
        if groupname_check and submissiontype_check and monthend_check:
            groupname_list += [temp_groupname]
            submissiontype_list += [temp_submissiontype]
            monthend_list += [temp_monthend]
            status += ['processed '+temp_groupname]
            prospectuses += [html_doc[html_doc.find('<TEXT>')+len('<TEXT>'):html_doc.find('</TEXT>')-1]]
        else:
            temp_status += temp_groupname+"."
            if groupname_check is False:
                temp_status += ' groupname unmatched.'
            if submissiontype_check is False:
                temp_status += ' submissiontype unmatched.'
            if monthend_check is False:
                temp_status += ' monthend unmatched.'
            status += [temp_status]
    for line in status:
        print (line)

    df = pd.DataFrame({'groupname' : groupname_list, 'submissiontype':submissiontype_list, 'monthend':monthend_list, 'prospectus':prospectuses})
    return df   


def create_connection(db_file):
    """ create a database connection to a SQLite database """
    try:
        conn = sqlite3.connect(db_file)
        return conn
    except Error as e:
        print(e)    
    return None

def create_table(conn):
    """ create a table in a SQLite database """
    try:
        # command to create sql table
        create_table_sql =  """CREATE TABLE FUND(  GROUPNAME             text     NOT NULL,
                                                   SUBMISSIONTYPE        text     NOT NULL,
                                                   MONTHEND              text     NOT NULL,
                                                   PROSPECTUS            text     NOT NULL
                                                );"""

        cursor = conn.cursor()
        cursor.execute(create_table_sql)        
    except Error as e:
        print(e)

def create_fund(conn, fund):
    sql = ''' INSERT INTO FUND(GROUPNAME,SUBMISSIONTYPE,MONTHEND,PROSPECTUS)
              VALUES(?,?,?,?) '''
    cur = conn.cursor()
    cur.execute(sql, fund)
    return cur.lastrowid

if __name__ == '__main__':
    #access to CSV
    doc_list = pd.read_csv("RA_test.csv")
    
    # Parse text from EDGAR links
    df = text_parser(doc_list)

    RA_db = create_connection("RA.db")
    create_table(RA_db)
    with RA_db:
        for i in range(len(df)):
            fund = (df.iloc[i]['groupname'],df.iloc[i]['submissiontype'],str(df.iloc[i]['monthend']),df.iloc[i]['prospectus'])
            print(fund[0])
            create_fund(RA_db,fund)
    RA_db.close()

processed wells fargo funds trust
failed strategic partners opportunity funds. submissiontype unmatched.
processed sti classic funds
processed ssga funds
processed mutual fund series trust
processed mosaic equity trust
wells fargo funds trust
sti classic funds
ssga funds
mutual fund series trust
mosaic equity trust


In [161]:
!ls

 RA_test.csv			   RAtest.ipynb		   sql.py   updated.py
'RA Test Instructions[8804].pdf'   RA_test_Tian_Zhang.py   t.db


In [0]:
!rm RA.db

In [0]:
def create_fund(conn, fund):
    sql = ''' INSERT INTO FUND (GROUPNAME,SUBMISSIONTYPE,MONTHEND,PROSPECTUS)
              VALUES(?,?,?,?) '''
    cur = conn.cursor()
    cur.execute(sql, fund)
    return cur.lastrowid

In [137]:
f = ('a','b','c','d')
create_fund(RA_db,f)

OperationalError: ignored

In [0]:
sql_insert = """INSERT INTO FUND (GROUPNAME,SUBMISSIONTYPE,MONTHEND,PROSPECTUS)
                VALUES(%s,%s,%s,%s)
                            """ %("a","b","c","d")

In [110]:
sql_insert

'INSERT INTO FUND(GROUPNAME,SUBMISSIONTYPE,MONTHEND,PROSPECTUS)\n                VALUES(a,b,c,d)\n                            '

In [0]:
connect = sqlite3.connect("RA.db")

In [0]:
c = connect.cursor()

In [128]:
d = "INSERT INTO FUND (GROUPNAME,SUBMISSIONTYPE,MONTHEND,PROSPECTUS) VALUES(a,b,c,d)"
c.execute(d)

OperationalError: ignored

In [0]:
def get_posts():
    with conn:
        cur.execute("SELECT * FROM Posts")
        print(cur.fetchall())